In [1]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
from DDPM import DiffusionModel
from utils import *
import numpy as np
import wandb
import random

In [7]:
img_size = 32
batch_size = 16
epochs = 1000
learning_rate = 0.0001
log = True

In [8]:
transform = transforms.Compose([
            transforms.Resize(img_size + int(.25*img_size)),     # Scale up by 25% to enable random crop
            transforms.RandomResizedCrop(img_size, scale=(0.8, 1.0)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # normalize mean and std for n=3 color channels
        ])

In [9]:
training_set = torchvision.datasets.CIFAR10('./data', train=True, transform=transform, download=True)
testing_set = torchvision.datasets.CIFAR10('./data', train=False, transform=transform, download=True)
training_set, validation_set = torch.utils.data.random_split(training_set, [45000, 5000])

Files already downloaded and verified
Files already downloaded and verified


In [10]:
training_loader = torch.utils.data.DataLoader(training_set, batch_size=batch_size, shuffle=True, num_workers=10,  pin_memory=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=batch_size, shuffle=True, num_workers=10, pin_memory=True)
testing_loader = torch.utils.data.DataLoader(testing_set, batch_size=batch_size, shuffle=True, num_workers=10, pin_memory=True)

In [11]:
model = DiffusionModel(
    learning_rate=learning_rate,
    in_channels=3,
    out_channels=3,
    noise_steps=1000,
    beta_start=1e-4,
    beta_end=0.02,
    img_size=img_size,
    device="cuda",
    num_class=10
)

In [ ]:
if log:
    wandb.init(
        project="DenoiseDiffusion",
        config={
        "dataset": "CIFAR-100",
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "epochs": epochs,
        "Type": "DDPM",
        "Save": "Save-0"
        }
    )

model.train(training_loader, validation_loader, epochs, log=log)
wandb.finish()

wandb: Currently logged in as: woodleighj (jackwoodleigh). Use `wandb login --relogin` to force relogin


Epoch 0... 


  0%|                                                                                         | 0/2813 [00:00<?, ?it/s]C:\Users\Jack\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
100%|████████████████████████████████████████████████████████████████████████████████| 313/313 [00:25<00:00, 12.07it/s]


Average Training Loss = 0.24715444009381599, Average Validation Loss = 0.13583948448919259

Model saved to diffusion_model.pth
Epoch 1... 


100%|████████████████████████████████████████████████████████████████████████████████| 313/313 [00:26<00:00, 12.03it/s]


Average Training Loss = 0.11280580135403347, Average Validation Loss = 0.09742596115881262

Model saved to diffusion_model.pth
Epoch 2... 


  0%|                                                                                         | 0/2813 [00:00<?, ?it/s]